In [1]:
#from models import *
import torch
import pytorchvideo
from pytorchvideo.data import labeled_video_dataset
from pytorchvideo.data.clip_sampling import make_clip_sampler
from pytorchvideo.transforms import ApplyTransformToKey, RandomResizedCrop, Normalize, RandomShortSideScale, RemoveKey, ShortSideScale, UniformTemporalSubsample
import torchvision.transforms as transforms 
from torch.utils.data import DataLoader
from tqdm import tqdm
import pytorchvideo.models as models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.models.video as v_model
from torch.utils.data import random_split


device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
leg = 'R'

In [2]:
model = v_model.r3d_18()
model = model.to(device)

checkpoint = torch.load(f'./hint_model_{leg}/right_model_6.pt' if leg == 'R' else f'./hint_model_{leg}/left_model_49.pt') 
model.load_state_dict(checkpoint)

model.eval()
print("model loaded!")

model loaded!


In [3]:
train_transform = transforms.Compose(
            [
            ApplyTransformToKey(
              key="video",
              transform = transforms.Compose(
                  [
                    #UniformTemporalSubsample(8),
                    #Lambda(lambda x: x / 255.0),
                    Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
                    #RandomShortSideScale(min_size=256, max_size=320),
                    #RandomCrop(244),
                    transforms.Resize((192,108)),
                    #transforms.RandomHorizontalFlip(p=0.5),
                  ]
                ),
              ),
            ]
        )

batch_size = 3
num_sec = 8

In [4]:
clip_sampler = make_clip_sampler('random', num_sec)
dataset = labeled_video_dataset(f'valid_set_{leg}', clip_sampler, transform=train_transform)
#train_set, valid_set = random_split(dataset, [int(len(dataset) * 0.8), len(dataset) - int(len(dataset) * 0.8)])
test_loader = DataLoader(dataset, batch_size = 3)
#valid_loader = DataLoader(valid_set, batch_size = 3, shuffle = False, num_workers = 0)

In [5]:
criterion = nn.CrossEntropyLoss()
valid_loss = 0
total = 0
accuracy = 0

with torch.no_grad():
    for batch in tqdm(test_loader):
            imgs, labels = batch['video'], batch['label']
            #print(batch['video_name'])
            #print(batch['clip_start'])
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            #print(preds, labels)

            # calculate loss
            loss = criterion(outputs, labels)

            # add loss
            valid_loss += loss.item()

            # add accuracy
            total += imgs.shape[0]
            accuracy += torch.sum(preds == labels.data)
print(f"test acc = {accuracy / total}, loss = {valid_loss / total}")

4it [00:11,  2.88s/it]

test acc = 0.4166666865348816, loss = 0.3619750291109085


In [ ]:
HAO SE O